# Решение #3 - Ансамбль

### Импортирование библиотек и загрузка датасета

In [1]:
from src import *
import pandas as pd
from tqdm.auto import tqdm
from ipywidgets import FloatProgress

df = pd.read_json('./../data/sample.json')

### Структура датасета

In [2]:
df.head()

,id,text
0,1,Ты нашёл их или нет?
1,2,Почему она так со мной поступает?
2,3,Никто туда больше не ходит.
3,4,У него с собой не было тогда денег.
4,5,Почему они с нами так поступают?


### Предобработка строк

In [3]:
df['text_preprocess'] = df['text'].apply(load_dataset.preprocess_text)

In [4]:
df.head()

,id,text,text_preprocess
0,1,Ты нашёл их или нет?,ты нашел их или нет
1,2,Почему она так со мной поступает?,почему она так со мной поступает
2,3,Никто туда больше не ходит.,никто туда больше не ходит
3,4,У него с собой не было тогда денег.,у него с собой не было тогда денег
4,5,Почему они с нами так поступают?,почему они с нами так поступают


### Решение основано на методе голосования, где каждый алгоритм голосует за один из ответов и каждый алгоритм имеет свой вес
Применяются следующие алгоритмы:
* Косинусное сходство с векторзацией подсчетом
* Косинусное сходство с векторизацией tf-idf
* Расстояние Левенштейна
* Расстояние Дамерау-Левенштейна
* Манхэттонское расстояние
* Евклидово расстояние
* Алгоритм выравнивания последовательности в сочетании с расстоянием Хэмминга
* Расстояние Хэмминга 
* Расстояние Хэллингера
* Индекс Жаккарда
* Расстояние Джаро-Винклера
* Дивергенция Дженсена-Шеннона
* Расстояние Карловского
* Алгоритм Майерса
* N-граммы
* N-граммы в сочетании с индексом Тверского
* Коэффициент Сёренсена
* Сравнение множеств слов
* Сравнение наборов местоимений

### Вспомогательный класс, который хранит указатель на функцию, пороговое значение и вес функции

In [5]:
class Function:
    def __init__(self,function,threshold:float,weight:int):
        self.function = function
        self.threshold = threshold
        self.weight = weight
    
    def vote(self,left,right):
        if self.threshold == 0:
            if self.function(left,right):
                return self.weight
            else:
                return -self.weight
        else:
            if self.function(left,right,self.threshold):
                return self.weight
            else:
                return -self.weight

In [ ]:
LOW = 1
MEDIUM = 2
HIGH = 3
EXTREME = 5

threshold = 0.85

functions = [Function(cosine_count.is_rewrite_cosine_count,threshold,MEDIUM),
            Function(cosine_tfidf.is_rewrite_cosine_tfidf,threshold,MEDIUM),
            Function(lev_dist.is_rewrite_levenshtein_distance,threshold,LOW),
            Function(dam_lev_dist.is_rewrite_damerau_levenshtein_distance,threshold,LOW),
            Function(distance_L1.is_rewrite_distance_L1,threshold,LOW),
            Function(distance_L2.is_rewrite_distance_L2,threshold,LOW),
            Function(hamming_distance_and_sequence_alignment_composition.is_rewrite_hamming_distance_normalized,threshold,LOW),
            Function(hamming_distance_custom.is_rewrite_hamming_distance_custom,threshold,LOW),
            Function(hellinger.is_rewrite_hellinger_distance,threshold,LOW),
            Function(jaccard.is_rewrite_Jaccard,threshold,MEDIUM),
            Function(jaro_winkler.is_rewrite_jaro_winkler,threshold,MEDIUM),
            Function(JSD.is_rewrite_JSD,threshold,MEDIUM),
            Function(karlovskiy_distance.is_rewrite_karlovskiy_distance,threshold,HIGH),
            Function(myers.is_rewrite_myers,threshold,LOW),
            Function(ngrams.is_rewrite_ngram,threshold,HIGH),
            Function(ngrams.is_rewrite_tverskiy_ngram,threshold,HIGH),
            Function(sorensen.is_rewrite_Sorensen,threshold,MEDIUM),
            Function(word_set.is_rewrite_word_set,threshold,MEDIUM),
            Function(pronouns.check_pronoun_correspondence,0,EXTREME)]

pd.options.mode.chained_assignment = None # Убираем лишние предупреждения
df['used'] = [False] * len(df)
groups = []

for i in tqdm(range(len(df))):
    if df['used'][i]:  # Проверка метки использования строки
        continue
    groups.append([])
    groups[-1].append(df['text'][i])
    df['used'][i] = True
    
    for j in range(i+1,len(df)):
        if df['used'][j]: # Проверка метки использования строки
            continue
            
        votes = sum([func.vote(df['text_preprocess'][i],df['text_preprocess'][j]) for func in functions])

        if(votes > 0):
            groups[-1].append(df['text'][j])
            df['used'][j] = True

    if len(groups[-1]) == 1: # Если строка не имеет рерайта, то группа не формируется
        df['used'][i] = False 
        groups = groups[:-1]


# Добавляем уникальные строки (не имеющие рерайт)
groups = groups + [[str] for str in list(df[df['used']==False]['text'])]

df = df.drop('used',axis=1)

  0%|          | 0/412 [00:00<?, ?it/s]

In [ ]:
assert(sum([len(group) for group in groups]) == len(df))

### Количество найденных групп

In [ ]:
len(groups)

### Все найденные группы

In [ ]:
groups

### Строки, которые имеют более 1 рерайта

In [ ]:
groups3 = [group for group in groups if len(group) > 2]

In [ ]:
len(groups3)

In [ ]:
groups3

### Строки, которые имеют ровно 1 рерайт

In [ ]:
groups2 = [group for group in groups if len(group) == 2]

In [ ]:
len(groups2)

In [ ]:
groups2

### Строки, которые не имеют рерайта

In [ ]:
groups1 = [group for group in groups if len(group) == 1]

In [ ]:
len(groups1)

In [ ]:
groups1

### Сохранение результата

In [ ]:
save_groups.save_groups(groups,"../output/solution3-result.json")